In [1]:
import os
import numpy as np
import scipy
import scipy.ndimage  
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set parameters
data_dir = 'dataset'
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

train_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Load base model
base_model = MobileNetV2(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet')
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train and save model
history = model.fit(train_gen, epochs=EPOCHS, validation_data=val_gen)
model.save("rice.h5")


Found 60000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.
Epoch 1/10
3750/3750 [==============================] - 1207s 321ms/step - loss: 0.1160 - accuracy: 0.9643 - val_loss: 0.0926 - val_accuracy: 0.9665
Epoch 2/10
3750/3750 [==============================] - 694s 185ms/step - loss: 0.0597 - accuracy: 0.9800 - val_loss: 0.0680 - val_accuracy: 0.9762
Epoch 3/10
3750/3750 [==============================] - 584s 156ms/step - loss: 0.0521 - accuracy: 0.9822 - val_loss: 0.0584 - val_accuracy: 0.9790
Epoch 4/10
3750/3750 [==============================] - 567s 151ms/step - loss: 0.0497 - accuracy: 0.9826 - val_loss: 0.0514 - val_accuracy: 0.9827
Epoch 5/10
3750/3750 [==============================] - 723s 193ms/step - loss: 0.0481 - accuracy: 0.9836 - val_loss: 0.0555 - val_accuracy: 0.9819
Epoch 6/10
3750/3750 [==============================] - 574s 153ms/step - loss: 0.0470 - accuracy: 0.9840 - val_loss: 0.0601 - val_accuracy: 0.9790
Epoch 7/10
3750/3750 [===

e:\rice_type_detection\rice-env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Print final training and validation accuracy
print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])


: 